In [1]:
import logging
import pathlib
from langchain_core.documents import Document
from langchain_community.vectorstores.docarray import DocArrayInMemorySearch
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.retrievers import BaseRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_loaders import (
  PyPDFLoader, TextLoader,
  UnstructuredWordDocumentLoader,
  UnstructuredEPubLoader
)
from langchain_community.vectorstores import docarray
from langchain.vectorstores.faiss import FAISS
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
#from langchain_openai import ChatOpenAI
from langchain.chains.base import Chain
from langchain.memory import ConversationBufferMemory
import streamlit as st
from streamlit.external.langchain import StreamlitCallbackHandler
from ctransformers import AutoModelForCausalLM
from langchain.llms import CTransformers
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from transformers import LlamaForCausalLM, LlamaTokenizer
import pickle
import os
import time
import tempfile


c:\Users\krish\Downloads\Jayant\Chatbot\ChatbotV1\ChatbotV1\chatbot1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"

In [2]:
from img2table.document import PDF
from img2table.ocr import TesseractOCR

# Instantiation of the pdf
pdf = PDF(src="data/publication1.pdf")

# Instantiation of the OCR, Tesseract, which requires prior installation
ocr = TesseractOCR(lang="eng")

# Table identification and extraction
pdf_tables = pdf.extract_tables(ocr=ocr)



In [3]:
pdf_tables

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: []}

In [3]:
from pdf2image import convert_from_path
pdf_path = "data/publication1.pdf"
images = convert_from_path(pdf_path,
                           poppler_path= r"C:\Program Files\poppler\Library\bin",
                           dpi=300)  # Convert each page to an image
for i, image in enumerate(images):
    image.save(f"page_{i}.png", "PNG")  # Save images for OCR



In [8]:
import pytesseract

table_text = pytesseract.image_to_string("page_5.png", config="--psm 6")  # Page segmentation mode 6 for tables
print(table_text)

BIOM 255 (Leffert) —- Discussion Feb. 1, 2007
between HER-2/neu amplification and nodal status (P = 0.002) 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66
indicated that information on amplification of this gene ma — — = tie 2 oe
correlate with disease behavior; that i recurrences and survival To — - - 2. ed * ba my B> ~ er
test this, univariate survival analyses were performed in which = WM
amplification was compared to relapse and survival in this patient =
group. A total of 35 patients had a recurrence of the disease, and 29
had died at the time of the analyses. Median times to relapse and
death were 62 months and 69 months, respectively. The median Fig. 4. roe of rchybridization of fiker with human EGER probe. Filters
follow-up time for patients still alive was 47 months, ranging from probe (2 B). in Fig’ tse ad Se ewer mo weight tk
24 to 86 months. A total of 71 of the 86 patients (83%) received hybridized with 22P-labeled EGFR probe in filter-containing lanes 51 to 66
some form o

In [7]:
import pytesseract

# Windows example
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Linux/Mac (if not in default location)
# pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

print(pytesseract.get_tesseract_version())

5.5.0.20241111


In [3]:
import numpy 
numpy.__version__

'2.1.0'

In [ ]:
# We can also create an excel file with the tables
pdf.to_xlsx('tables.xlsx',
            ocr=ocr)